<a href="https://colab.research.google.com/github/Yash-Kamtekar/Optional_Assignment/blob/main/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First import the libraries which will be useful.

In [ ]:
import csv
from itertools import combinations
from google.colab import drive

mounting drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


# Read the data from csv file and store in dict.

In [ ]:
def read_data(file_loc='/content/drive/MyDrive/255/Optional_Assignment_1/groceries.csv'):
    trans = dict()
    with open(file_loc) as f:
        filedata = csv.reader(f, delimiter=',')
        count = 0
        for line in filedata:
            count += 1
            trans[count] = list(set(line))
    return trans

# Count the frequency of each item.

In [ ]:
def frequence(items_lst, trans, check=False):
    items_counts = dict()
    for i in items_lst:
        temp_i = {i}
        if check:
            temp_i = set(i)
        for j in trans.items():
            if temp_i.issubset(set(j[1])):
                if i in items_counts:
                    items_counts[i] += 1
                else:
                    items_counts[i] = 1
    return items_counts

# Form association rules form support.

In [ ]:
def association_rules(items_grater_then_min_support):
    rules = []
    dict_rules = {}
    for i in items_grater_then_min_support:
        dict_rules = {}
        if type(i) != type(str()):
            i = list(i)
            temp_i = i[:]
            for j in range(len(i)):
                k = temp_i[j]
                del temp_i[j]
                dict_rules[k] = temp_i
                temp_i = i[:]
        rules.append(dict_rules)
    temp = []
    for i in rules:
        for j in i.items():
            if type(j[1]) != type(str()):
                temp.append({tuple(j[1])[0]: j[0]})
            else:
                temp.append({j[1]: j[0]})
    rules.extend(temp)
    return rules

# Find the confidence of those association rules and take only rules which are greater than the minimum confidence.

In [ ]:
def confidence(associations, d, min_confidence):
    ans = {}
    for i in associations:
        for j in i.items():
            if type(j[0]) == type(str()):
                left = {j[0]}
            else:
                left = set(j[0])
            if type(j[1]) == type(str()):
                right = {j[1]}
            else:
                right = set(j[1])

            for k in d:
                if type(k) != type(str()):
                    if left.union(right) - set(k) == set():
                        up = d[k]
                    if len(right) == len(set(k)) and right - set(k) == set():
                        down = d[k]
                else:
                    if len(right) >= len({k}):
                        if right - {k} == set():
                            down = d[k]
                        elif len(right) <= len({k}):
                            if {k} - right == set():    
                                down = d[k]
            if up/down >= min_confidence:
                ans[tuple(left)[0]] = right, up/down, up, down
    return ans

# Here is our main function that operates above code and here you can change the minimum support and confidence.

In [ ]:
def main(min_support, min_confidence, file_loc):
    trans = read_data()
    number_of_trans = [len(i) for i in trans.values()]
    items_lst = set()

    itemcount_track = list()
    
    for i in trans.values():
        for j in i:
            items_lst.add(j)

    store_item_lst = list(items_lst)[:]
    items_grater_then_min_support = list()
    items_counts = frequence(items_lst, trans)
    itemcount_track.append(items_counts)
    items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})

    for i in range(2, max(number_of_trans)+1):
        item_list = combinations(items_lst, i)
        items_counts = frequence(item_list, trans, check=True)
        itemcount_track.append(items_counts)
        if list({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support}.keys()) != []:
            items_grater_then_min_support.append({j[0]:j[1] for j in support(items_counts, trans).items() if j[1]>min_support})

    d = {}
    {d.update(i) for i in itemcount_track}
    associations = association_rules(items_grater_then_min_support[len(items_grater_then_min_support)-1])
    associations_grater_then_confidene = confidence(associations, d, min_confidence)

    print(associations_grater_then_confidene)

main(0.01, 0.7, '/content/drive/MyDrive/255/Optional_Assignment_1/groceries.csv')

NameError: ignored

In [ ]:
from collections import Counter
from itertools import combinations
import numpy as np

transactions = {
    1: ["1", "3", "4"],
    2: ["2", "3", "5"],
    3: ["1", "2", "3", "5"],
    5: ["2", "5"],
    6: ["1", "3", "5"]
}

min_support_count = 2
min_confidence_value = 0.6


# apriori pruning concept
def _pruning(current, previous, size):
    final_keys = []
    previous = [tuple(i) for i in previous]
    for key in current:
        FLAG = False
        current_comb = list(combinations(key, size))
        for i in current_comb:
            if i in previous or i[::-1] in previous:
                FLAG = True
            else:
                FLAG = False
                break

        if FLAG:
            final_keys.append(key)

    return final_keys


def support_value(itemset_keys_, transactions):
    itemset = {key: 0 for key in itemset_keys_}

    for keys in itemset_keys_:
        for val in transactions.values():
            if set(keys) & set(val) == set(keys):
                itemset[keys] += 1
    return itemset


# creating frequent itemset
def get_frequent_itemset(size=None, transactions=None, itemset=None):
    if size == 1:
        itemset = Counter()

        for val in transactions.values():
            itemset.update(val)

    else:

        prev_itemset_keys = list(itemset.keys())
        prev_itemset = itemset.copy()

        valid_keys = list(set(itemset.keys()))
        # flatten list of tuple -> keys: [(), ()] -> []
        # useful for running a combination of all the chosen features
        l = []
        for row in valid_keys:
            l.extend(row)

        valid_keys = set(l)

        # candidate itemset keys
        itemset_keys_ = list(combinations(valid_keys, size))

        # Apriori algorithm is based on theconcept that a subset
        # of a frequent itemset must also be a frequent itemset
        # so we are pruning away those features whose subset are not present
        # in the previous frequent itemset
        if size >= 2:
            itemset_keys_ = _pruning(
                itemset_keys_, prev_itemset_keys, size - 1)

        # finding support value for each of the selected itemset feature combination
        itemset = support_value(itemset_keys_, transactions)

        # defaulting back to th previous frequent itemset if
        # the iteration doesn't find any itemset which has the theshold required
        if itemset == {}:
            itemset = prev_itemset

    # getting frequent itemset from itemset
    # Frequent Itemset is an itemset whose support
    # value is greater than a threshold value(support).

    frequent_itemset = {}
    for key, val in itemset.items():
        if val >= min_support_count:
            frequent_itemset[key] = val

    return frequent_itemset


def finding_subsets(frequent_set):
    item_list = []
    size = len(list(frequent_set.keys())[0])
    for key in frequent_set.keys():
        subsets = []
        for i in range(1, size):
            subsets.append(list(combinations(key, i)))

        subsets = list(np.array(subsets).flatten())
        subsets.insert(0, key)
        item_list.append(subsets)

    return item_list


def finding_rules(itemset_sub):
    print("Antecedents -->  Consequents --- Confidence")
    for i in range(1, len(itemset_sub)):

        # passing as list as we have designed support_value function as
        # a function that takes an iteratable list of itemsets
        x = support_value([itemset_sub[0], ], transactions)
        y = support_value([itemset_sub[i], ], transactions)
        confidence = list(x.values())[0] / list(y.values())[0]
        if confidence >= min_confidence_value:
            print(
                f"{itemset_sub[i]} --> {itemset_sub[0]} --- {round(confidence, 2)}")


print("""
    ITEMS
    1: Banana
    2: Eggs
    3: Milk
    4: Tea
    5: Bread
""")

f = {}

for i in range(1, 5):
    f = get_frequent_itemset(size=i, transactions=transactions,
                             itemset=f)

# frequent_itemsets

print("Frequent Itemsets...")
for key, val in f.items():
    print(f"Itemset: {key}, support value: {val}")


subset = finding_subsets(f)

for i in subset:
    print(f"Rules for itemset - {i[0]}")
    finding_rules(i)
    print()


    ITEMS
    1: Banana
    2: Eggs
    3: Milk
    4: Tea
    5: Bread

Frequent Itemsets...
Itemset: ('3', '1', '5'), support value: 2
Itemset: ('3', '2', '5'), support value: 2
Rules for itemset - ('3', '1', '5')
Antecedents -->  Consequents --- Confidence
('1',) --> ('3', '1', '5') --- 0.67
('3', '1') --> ('3', '1', '5') --- 0.67
('3', '5') --> ('3', '1', '5') --- 0.67
('1', '5') --> ('3', '1', '5') --- 1.0

Rules for itemset - ('3', '2', '5')
Antecedents -->  Consequents --- Confidence
('2',) --> ('3', '2', '5') --- 0.67
('3', '2') --> ('3', '2', '5') --- 1.0
('3', '5') --> ('3', '2', '5') --- 0.67
('2', '5') --> ('3', '2', '5') --- 0.67



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:120: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
